<font color=gray>Oracle Cloud Infrastructure Data Science Sample Notebook

Copyright (c) 2021, 2023 Oracle, Inc.  All rights reserved. <br>
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

# Uploading Larger Size Model Artifact Using Oracle ADS Library 

This notebook demonstrates simple solution for Oracle ADS Library which allows data scientists to upload larger model artifacts and eliminate the timeout error that is experienced by most folks when the artifact is large. It shows end-to-end steps from setting up the configuration till uploading the model artifact.

## Pre-requisites to Running this Notebook 

* We recommend that you run this notebook in a notebook session using a conda environment that has ADS version 2.8.10 installed

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import ads
import os

In [6]:
# Switch to 'api_key' if you prefer user principal (config + key file) as a method of authentication. 
# In this notebook, we are using resource principal. 
ads.set_auth(auth='resource_principal')

In [ ]:
seed = 42
# make some classification data
X, y = make_classification(n_samples=10000, n_features=15, n_classes=2, flip_y=0.05)
trainx, testx, trainy, testy = train_test_split(X, y, test_size=30, random_state=seed)
model = RandomForestClassifier(
        n_estimators=100, random_state=42
    )
# train a random forest classifier
model.fit(
        trainx,
        trainy,
    )

In [ ]:
# Prepare the model artifacts
from ads.model.framework.sklearn_model import SklearnModel
from ads.common.model_metadata import UseCaseType

sklearn_model = SklearnModel(estimator=model, artifact_dir="~/sklearn_artifact_dir")
sklearn_model.prepare(
    inference_conda_env="generalml_p38_cpu_v1",
    training_conda_env="generalml_p38_cpu_v1",
    X_sample=trainx,
    y_sample=trainy,
    use_case_type=UseCaseType.BINARY_CLASSIFICATION,
)

In [ ]:
# Saving the model artifact to the model catalog:
mc_model = sklearn_model.save(project_id=os.environ['PROJECT_OCID'],
                                  compartment_id=os.environ['NB_SESSION_COMPARTMENT_OCID'],
                                  training_id=os.environ['NB_SESSION_OCID'],
                                  display_name="<replace-with-your-display-name>",
                                  description="<replace-with-description>",
                                  ignore_pending_changes=True,
                                  ignore_introspection=True,
                                  bucket_uri="oci://<replace-with-your-bucket-name>",
                                 )
mc_model